## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chokurdakh,RU,70.6333,147.9167,-48.71,100,67,2.35,broken clouds
1,1,Mbarara,UG,-0.6080,30.6500,64.94,89,94,2.93,light rain
2,2,Esperance,AU,-33.8667,121.9000,58.84,76,0,12.50,clear sky
3,3,Dingtao,CN,35.0683,115.5625,46.11,83,100,5.06,overcast clouds
4,4,Hermanus,ZA,-34.4187,19.2345,70.90,68,86,20.65,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
16,16,Rikitea,PF,-23.1203,-134.9692,75.27,73,100,13.80,overcast clouds
19,19,Samarai,PG,-10.6167,150.6667,81.10,80,97,11.45,overcast clouds
21,21,Castro,BR,-24.7911,-50.0119,83.50,32,75,6.20,broken clouds
26,26,Harper,LR,4.3750,-7.7169,83.08,72,96,11.05,overcast clouds
27,27,Bengkulu,ID,-3.8004,102.2655,76.55,88,96,3.60,light rain
28,28,Puerto Ayora,EC,-0.7393,-90.3518,78.76,91,100,7.72,overcast clouds
29,29,Lorengau,PG,-2.0226,147.2712,81.86,85,100,12.82,overcast clouds
30,30,Poum,NC,-20.2333,164.0167,81.68,78,100,14.72,overcast clouds
32,32,Georgetown,MY,5.4112,100.3354,82.89,76,20,1.01,thunderstorm
35,35,Gigmoto,PH,13.7796,124.3900,78.80,90,100,14.74,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Country                199
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
16,16,Rikitea,PF,-23.1203,-134.9692,75.27,73,100,13.80,overcast clouds
19,19,Samarai,PG,-10.6167,150.6667,81.10,80,97,11.45,overcast clouds
21,21,Castro,BR,-24.7911,-50.0119,83.50,32,75,6.20,broken clouds
26,26,Harper,LR,4.3750,-7.7169,83.08,72,96,11.05,overcast clouds
27,27,Bengkulu,ID,-3.8004,102.2655,76.55,88,96,3.60,light rain
28,28,Puerto Ayora,EC,-0.7393,-90.3518,78.76,91,100,7.72,overcast clouds
29,29,Lorengau,PG,-2.0226,147.2712,81.86,85,100,12.82,overcast clouds
30,30,Poum,NC,-20.2333,164.0167,81.68,78,100,14.72,overcast clouds
32,32,Georgetown,MY,5.4112,100.3354,82.89,76,20,1.01,thunderstorm
35,35,Gigmoto,PH,13.7796,124.3900,78.80,90,100,14.74,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Rikitea,PF,75.27,overcast clouds,-23.1203,-134.9692,
19,Samarai,PG,81.10,overcast clouds,-10.6167,150.6667,
21,Castro,BR,83.50,broken clouds,-24.7911,-50.0119,
26,Harper,LR,83.08,overcast clouds,4.3750,-7.7169,
27,Bengkulu,ID,76.55,light rain,-3.8004,102.2655,
28,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,
29,Lorengau,PG,81.86,overcast clouds,-2.0226,147.2712,
30,Poum,NC,81.68,overcast clouds,-20.2333,164.0167,
32,Georgetown,MY,82.89,thunderstorm,5.4112,100.3354,
35,Gigmoto,PH,78.80,light rain,13.7796,124.3900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]   
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Rikitea,PF,75.27,overcast clouds,-23.1203,-134.9692,People ThankYou
19,Samarai,PG,81.10,overcast clouds,-10.6167,150.6667,Nuli Sapi
21,Castro,BR,83.50,broken clouds,-24.7911,-50.0119,CHACARA BAILLY
26,Harper,LR,83.08,overcast clouds,4.3750,-7.7169,Screensaver
27,Bengkulu,ID,76.55,light rain,-3.8004,102.2655,Grage Hotel Bengkulu
28,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
29,Lorengau,PG,81.86,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
30,Poum,NC,81.68,overcast clouds,-20.2333,164.0167,Gîte kajeon
32,Georgetown,MY,82.89,thunderstorm,5.4112,100.3354,Cititel Penang
35,Gigmoto,PH,78.80,light rain,13.7796,124.3900,Cj's Travellers Inn


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))